In [0]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression

In [0]:
spark = SparkSession.builder.appName('b').getOrCreate()

In [0]:
df=spark.read.csv('/FileStore/tables/customer_churn.csv',header=True,inferSchema=True)

In [0]:
df.columns

Out[25]: ['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [0]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler,OneHotEncoder,VectorIndexer
from pyspark.ml.feature import StringIndexer

final_df = df.select(['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Churn']).na.drop()
final_df.show()

+----+--------------+---------------+-----+---------+-----+
 Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn|
+----+--------------+---------------+-----+---------+-----+
42.0| 11066.8| 0| 7.22| 8.0| 1|
41.0| 11916.22| 0| 6.5| 11.0| 1|
38.0| 12884.75| 0| 6.67| 12.0| 1|
42.0| 8010.76| 0| 6.71| 10.0| 1|
37.0| 9191.58| 0| 5.56| 9.0| 1|
48.0| 10356.02| 0| 5.12| 8.0| 1|
44.0| 11331.58| 1| 5.23| 11.0| 1|
32.0| 9885.12| 1| 6.92| 9.0| 1|
43.0| 14062.6| 1| 5.46| 11.0| 1|
40.0| 8066.94| 1| 7.11| 11.0| 1|
30.0| 11575.37| 1| 5.22| 8.0| 1|
45.0| 8771.02| 1| 6.64| 11.0| 1|
45.0| 8988.67| 1| 4.84| 11.0| 1|
40.0| 8283.32| 1| 5.1| 13.0| 1|
41.0| 6569.87| 1| 4.3| 11.0| 1|
38.0| 10494.82| 1| 6.81| 12.0| 1|
45.0| 8213.41| 1| 7.35| 11.0| 1|
43.0| 11226.88| 0| 8.08| 12.0| 1|
53.0| 5515.09| 0| 6.85| 8.0| 1|
46.0| 8046.4| 1| 5.69| 8.0| 1|
+----+--------------+---------------+-----+---------+-----+
only showing top 20 rows

In [0]:
assembler = VectorAssembler( inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [0]:
from pyspark.ml import Pipeline 

log_reg_churn = LogisticRegression(featuresCol='features',labelCol='Churn')
pipeline = Pipeline(stages=[assembler,log_reg_churn])
#output = assembler.transform(final_df)

In [0]:
output.head(1)

Out[43]: [Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_line_cat=16.0, features=DenseVector([16.0, 6.0, 30.277, 6.94, 5.94, 3.55, 42.64]))]

In [0]:
train_data,test_data = final_df.randomSplit([0.7,0.3])

In [0]:
lr_model = pipeline.fit(train_data)

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
my_Eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Churn')
results = lr_model.transform(test_data)
results.select('prediction','Churn')
AUC=my_Eval.evaluate(results)

In [0]:
AUC

Out[34]: 0.7582977714556662

In [0]:
new_data=spark.read.csv('/FileStore/tables/new_customers.csv',header=True,inferSchema=True)

In [0]:
final_new_df = new_data.select(['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites','Company']).na.drop()
output_new = assembler.transform(final_new_df)

In [0]:
output = assembler.transform(final_df)
final_df = output.select('features','Churn')
final_model=log_reg_churn.fit(final_df)

In [0]:
new_results = final_model.transform(output_new)

In [0]:
new_results.select('Company','prediction').show()

+----------------+----------+
 Company|prediction|
+----------------+----------+
 King Ltd| 0.0|
 Cannon-Benson| 1.0|
Barron-Robertson| 1.0|
 Sexton-Golden| 1.0|
 Wood LLC| 0.0|
 Parks-Robbins| 1.0|
+----------------+----------+